## Modely na spracovanie signálov z vibrácií
**Autor:** Miroslav Hájek

**Predmet**: Bakalárska práca

Modely na monitorovanie vibrácií na základe analyzovaných algoritmov detekcie špičiek. Meranie úspešnosti na synteticky generovaných dátach.

### Toky v návrhu riešenia
1. Frekvenčná analýza
2. Online štatistiky v čase
3. Podvzorkovanie

#### Algoritmy na rozpoznávanie špičiek
za aplikovania vyhladzovania cez priemerovací filter

0. Detekcia špičiek prahovou úrovňou
1. Význačnosť vrchola spomedzi susedov
2. Algoritmus prechodu nulou do záporu
3. Algoritmus horského turistu

#### Štatistiky v časovej doméne (Posuvné okno)
- Amplitúda špička-špička
- Kvadratický priemer (RMS)
- Výberový priemer
- Medián, Horný a dolný kvartil
- Výberový rozptyl
- Smerodajná odchýlka
- Priemerná absolútna odchýlka 
- Mediánová absolútna odchýlka 
- Medzikvartilové rozpätie
- Korelácia

#### Rozklad vo frekvenčnej doméne
- Fouriérová transformácia - FFT
- DCT-II, DCT-IV
- Oknové funkcie: obdĺžníkové, Bartlettovo, Hannovo, Hammingovo a Blackmanovo okno
- Welchova metóda

#### Numerická integrácia pre získanie rýchlosti a polohy
- Obdlžnikové, Lichobežníkovo, Simsonovo pravidlo na úsekoch
- Korekcia obálkami: B-spline / Amplitúdová obálka (ako mean filter)

In [ ]:
import numpy as np
import scipy.fft
from scipy.signal.windows import (
    boxcar,
    bartlett,
    hann,
    hamming,
    blackman,
    gaussian
)
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from typing import Tuple
import json
from functools import partial

TimeSeries = Tuple[np.array, np.array]
gauss = partial(gaussian, std=1)

### Detekcia špičiek prahovou úrovňou
- `y`: postupnosť vzoriek
- `t`: prahová úroveň. Rozsah: $[0, \max{\{y\}}]$

In [ ]:
def find_peaks_threshold(y: np.array, t: float):
    return np.where(y >= t)

### Význačnosť vrchola spomedzi susedov
- `y`: postupnosť vzoriek
- `k`: počet najbližších uvažovaných susedov na každú zo strán od vrchola $[i-k; i+k]$. Rozsah: $[1, \mathrm{length}(y)\,/\, 2]$
- `e`: relatívna tolerancia vyššieho bodu v susedstve od vrchola. Rozsah: $[0, \max{\{y\}}]$
- `h_rel`: relatívna výška vrcholu v susedstve. Rozsah: $[0, \max{\{y\}}]$
- `h`: Absolútna amplitúda. Rozsah: $[0, \max{\{y\}}]$

In [ ]:
def find_peaks_neighbours(y: np.array, k: int, e: float, h_rel: float, h: float) -> list:
    peaks = []
    for i in range(len(y)):
        if abs(y[i]) >= h:
            possible_peak = True
            a = max(i - k, 0)
            b = min(i + k, len(y))
            valley = y[a]

            for j in range(a, b):
                if i != j:
                    if y[j] - y[i] > e:
                        possible_peak = False
                    if y[j] < valley:
                        valley = y[j]

            if possible_peak is True and y[i] - valley > h_rel:
                peaks.append(i)
    return peaks

### Algoritmus prechodu nulou do záporu
- `y`: postupnosť vzoriek
- `k`: dlžka sečnice $[i-k; i+k]$. Rozsah: $[1, \mathrm{length}(y)\,/\, 2]$
- `slope`: prahová strmosť kopca. $slope >= 0$

In [ ]:
def find_peaks_zero_crossing(y: np.array, k: int, slope: float) -> list:
    peaks = []

    for i in range(k, len(y) - k):
        if ((y[i + k] - y[i]) < 0 and (y[i] - y[i - k]) > 0 and
                abs(y[i + k] - y[i]) >= slope and abs(y[i] - y[i - k]) >= slope):
            peaks.append(i)

    return peaks

### Algoritmus horského turistu
- `y`: postupnosť vzoriek
- `tolerance`: prahová úroveň vo vertikálnej osi. Rozsah: $[0, \max{\{y\}}]$
- `hole`: prahová úroveň v horizontálnej osi. Rozsah: $[0, \mathrm{length}(y)]$
- `prominence`: relatívna výška oproti predošlej navštívenej doline. Rozsah: $[0, \max{\{y\}}]$
- `isolation`: vzdialenosť k najbližšiemu skoršiemu vrcholu. Rozsah: $[0, \mathrm{length}(y)]$

In [ ]:
def find_peaks_hill_walker(
        y: np.array, tolerance: float, hole: float, prominence: float, isolation: float) -> list:
    peaks = []
    i_change = 0
    y_valley = 0
    possible_change = False
    uphill = bool((y[1] - y[0]) >= 0)
    
    for i in range(1, len(y)):
        y_step = y[i] - y[i - 1]
        slope = bool(y_step >= 0)

        if possible_change is False and uphill != slope:
            possible_change = True
            i_change = i - 1
        elif possible_change is True and uphill == slope:
            possible_change = False
        
        if (possible_change is True and
                uphill != slope and
                abs(i - i_change) > hole and
                abs(y[i] - y[i_change]) > tolerance):
            
            possible_change = False
            prev_uphill = uphill
            uphill = slope

            if prev_uphill is False and uphill is True:
                y_valley = y[i_change]

            elif (prev_uphill is True and 
                      uphill is False and 
                      abs(y[i - hole] - y_valley) > prominence):
                if len(peaks) < 1 or (len(peaks) >= 1 and i_change - peaks[-1] > isolation):
                    y_peak = y[i_change]
                    peaks.append(i_change)
                  
    return peaks

### Generovanie sínusoidy
s frekvenciou, trvaním, pri vzorkovacej frekvencii

In [ ]:
def sine_generate(f: int, t: float, fs: int) -> TimeSeries:
    N = fs * t                                         # počet vzoriek
    Ts = 1 / fs                                        # interval vzorkovania
    t_series = Ts * np.arange(N)
    return t_series, np.sin(2 * np.pi * f * t_series)  # y = A * sin(2*pi*t)

def plot_sinusoid(x, y, p, ax):
    ax.plot(x, y)
    if len(p) > 0: 
        ax.plot(x[p], y[p], 'o', color='r')
    ax.grid()

In [ ]:
x, y = sine_generate(10, 1, 200)
fig, axes = plt.subplots(3, 1, figsize=(20, 10))

p = find_peaks_neighbours(y, k=2, e=0, h_rel=0, h=0)
print('find_peaks_neighbours:\t', p)
plot_sinusoid(x, y, p, axes[0])

p = find_peaks_zero_crossing(y, k=1, slope=0)
print('find_peaks_zero_crossing:', p)
plot_sinusoid(x, y, p, axes[1])

p = find_peaks_hill_walker(y, tolerance=0, hole=0, prominence=0, isolation=0)
print('find_peaks_hill_walker:\t', p)
plot_sinusoid(x, y, p, axes[2])

### Oknovanie (1) a transformácia do frekvenčnej domény (2)
- `y`: postupnosť vzoriek
- `t`: časové značky v sekundách pre jednotlivé vzorky
- `window`: oknová funkcia. Rozsah: boxcar, bartlett, hann, hamming, blackman
- `Wn`: dĺžka posuvného okna v počte vzoriek. Rozsah: $[1, \mathrm{length}(y)]$
- `overlap`: pomer prelínania posuvných okien. Rozsah: $[0, 0.99]$
- `fs`: vzorkovacia frekvencia v Hz
- `mode`: spôsob transformácie magnitúdového spektra (linear vs. log)

In [ ]:
def fft_transform(v):
    elements = np.fft.fft(v)
    return np.absolute(elements)[:len(elements)//2]

def fft_log_transform(v):
    yf = fft_transform(v)
    return 20 * np.log10(yf / yf.max())

def rfft_transform(v):
    elements = np.fft.rfft(v)
    return np.absolute(elements)[:-1]

def rfft_log_transform(v):
    yf = rfft_transform(v)
    return 20 * np.log10(yf / yf.max())

def dctii_transform(v):
    elements = scipy.fft.dct(np.array(v), type=2)
    return np.absolute(elements)[:len(elements)//2]

def dctii_log_transform(v):
    yf = dctii_transform(v)
    return 20 * np.log10(yf / yf.max())

def dctiv_transform(v):
    elements = scipy.fft.dct(np.array(v), type=4)
    return np.absolute(elements)[:len(elements)//2]

def dctiv_log_transform(v):
    yf = dctiv_transform(v)
    return 20 * np.log10(yf / yf.max())

In [ ]:
def frequency_spectrum(
        t: np.array, 
        y: np.array, 
        fs: int, 
        window,
        Wn: int,
        overlap: float,
        trans='fft', 
        mode='linear'
    ):
    step = Wn - int(Wn * overlap)
    Wi = np.arange(0, len(y) - Wn, step)
    
    if trans == 'fft':
        transform = fft_transform
        if mode == 'log':
            transform = fft_log_transform
    elif trans == 'rfft':
        transform = rfft_transform
        if mode == 'log':
            transform = rfft_log_transform
    elif trans == 'dct-ii':
        transform = dctii_transform
        if mode == 'log':
            transform = dctii_log_transform
    elif trans == 'dct-iv':
        transform = dctiv_transform
        if mode == 'log':
            transform = dctiv_log_transform
    
    win = window(Wn)
    x = t[Wi]
    f = np.fft.fftfreq(Wn, 1/fs)[:Wn//2]
    f_mag = np.asarray([
        transform(y[i:i+Wn] * win) for i in Wi
    ])
     
    return {
        'fs': fs,
        'Wn': Wn,
        'overlap': overlap,
        'resolution': fs / Wn,
        'bins': f,
        't_windows': x,
        'magnitudes': f_mag
    }

### Hľadanie špičiek v každom posuvnom okne

In [ ]:
def peaks_indices_windowed(peak_find, magnitudes):
    return [peak_find(window) for window in magnitudes]

def peak_freqs_windowed(i_peaks, bins, resolution):
    return [list(zip(bins[p], bins[p] + resolution)) for p in i_peaks] 

### Vykreslenie grafu spektrálnej magnitúdy pre posuvné okná zvlášť

In [ ]:
def plot_spectra_slices(bins, magnitudes, peaks=[]):
    fig, axes = plt.subplots(len(magnitudes), 1, figsize=(20, 10))
    for i, amp in enumerate(magnitudes):
        p = peaks[i] if len(peaks) > i else []
        plot_sinusoid(bins, amp, p, axes[i])
        # axes[i].set_xlim(-5, 5)


def plot_spectrogram(spectrum, ax, ylog, yrange):
    if ylog is True:
        ax.set_yscale('log', base=2)
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
        plt.ticklabel_format(axis='y', style='plain')
        
    if yrange is not None:
        ax.set_ylim(*yrange)

    ax.pcolormesh(spectrum['t_windows'], spectrum['bins'], spectrum['magnitudes'].T, shading='gouraud')
    ax.set_xlabel('Čas [s]')
    ax.set_ylabel('Frekvencia [Hz]')
    ax.grid(True)


def plot_spectra_heatmap(spectrum, peaks=[], ylog=False, yrange=None):
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    plot_spectrogram(spectrum, ax, ylog, yrange)
    
    for i, p in enumerate(peaks):
        t = spectrum['t_windows'][i]
        f = spectrum['bins'][p]
        for pos in f:
            ax.plot(t, pos, 'o', color='w', alpha=0.5)


def plot_spectra_events(spectrum, events, ylog=False, yrange=None):
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    plot_spectrogram(spectrum, ax, ylog, yrange)
    
    for event in events:
        if event['action'] == 'END':
            t_start = spectrum['t_windows'][event['start']]
            t_end = spectrum['t_windows'][event['start'] + event['duration'] - 1]
            f = event['frequency'] 
            ax.plot((t_start, t_end), (f, f), color='r')


### Prepočet poradia frekvenčného vedierka na frekvencie v Hz pre skutočné a predikované špičky

In [ ]:
def found_peak_freqs(truth, peaks, spectrum):
    f_truth = peak_freqs_windowed(truth, spectrum['bins'], spectrum['resolution'])
    f_peaks = peak_freqs_windowed(peaks, spectrum['bins'], spectrum['resolution'])

    for t_win, p_win in zip(f_truth, f_peaks):
        print("Okno:")
        print('\tI:', ', '.join([f'<{a:.2f}-{b:.2f}>' for a, b in t_win]))
        print('\tO:', ', '.join([f'<{a:.2f}-{b:.2f}>' for a, b in p_win]))  

### Metriky pre binárny klasifikátor pre okná zvlášť, s mikro a makro priemerovaním

In [ ]:
def ground_truth(rules, windows, resolution):
    labels = []
    for start in windows:
        freqs = []
        for sine in rules.get('sines', []):
            if (sine['t0'] <= start <= sine['tn'] or 
                    sine['t0'] <= start + spectrum['Wn'] <= sine['tn']):
                freqs.append(int(sine['freq'] / resolution))
        labels.append(freqs)
    return labels

def metrics(bins, y_truth, y_predict):
    stats = []

    for win_truth, win_predict in zip(y_truth, y_predict):
        truth = np.full(len(bins), False)
        predict = np.full(len(bins), False)
        
        truth[win_truth] = True
        predict[win_predict] = True
        
        tp = sum([int(x and y) for x, y in zip(truth, predict)])
        tn = sum([int(not x and not y) for x, y in zip(truth, predict)])
        fp = sum([int(not x and y) for x, y in zip(truth, predict)])
        fn = sum([int(x and not y) for x, y in zip(truth, predict)])
        
        tpr = tp / (tp + fn)
        tnr = tn / (tn + fp)
        ppv = 0
        if tp + fp != 0:
            ppv = tp / (tp + fp)
        acc = (tp + tn) / (tp + tn + fp + fn)
        fpr = fp / (fp + tn)
            
        stats.append({
            'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn, 
            'TPR': tpr, 'TNR': tnr,  'PPV': ppv,
            'ACC': acc, 'FPR': fpr
        })

    return stats

def metric_macro_average(metrics):
    averages = {
        'TPR': 0, 'TNR': 0,  'PPV': 0, 'ACC': 0, 'FPR': 0
    }
    for win_metrics in metrics:
        for k in averages.keys():
            averages[k] += win_metrics[k]
        
    for k in averages.keys():
        averages[k] /= len(metrics)
        
    return averages


def metric_micro_average(metrics):
    averages = {
        'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0, 
    }
    for win_metrics in metrics:
        for k in averages.keys():
            averages[k] += win_metrics[k]

    tp, tn, fp, fn = (
        averages['TP'], averages['TN'], averages['FP'], averages['FN']
    )
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    ppv = 0
    if tp + fp != 0:
        ppv = tp / (tp + fp)
    acc = (tp + tn) / (tp + tn + fp + fn)
    fpr = fp / (fp + tn)
    
    return {
        'TPR': tpr, 'TNR': tnr, 'PPV': ppv,
        'ACC': acc, 'FPR': fpr
    }

### Generovanie sinusoidy s f = 15 Hz, trvaním t = 1s pri fs = 250Hz

In [ ]:
plt.rcParams['figure.figsize'] = (10, 3)
rec = {
    'duration': 1,
    'f_sampling': 250,
    'sines': [{'freq': 15, 'amp': 1, 't0': 0, 'tn': 1}],
}
t, y = sine_generate(f=rec['sines'][0]['freq'], t=rec['duration'], fs=rec['f_sampling'])
plt.plot(t, y)
plt.show()

### Hľadanie špičiek spomedzi najbližších susedov vo frekvenčnej doméne na sínusoide s f = 20 Hz

In [ ]:
spectrum = frequency_spectrum(t, y, fs=rec['f_sampling'], window=hann, Wn=63, overlap=0, mode='linear')
func = partial(find_peaks_neighbours, k=3, e=0, h_rel=0, h=0)

peaks = peaks_indices_windowed(func, spectrum['magnitudes'])
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(rec, spectrum['t_windows'], spectrum['resolution'])
found_peak_freqs(truth, peaks, spectrum)

### Vyhodnotenie klasifikácie špičiek

In [ ]:
m = metrics(spectrum['bins'], truth, peaks)
from pprint import pprint
print('Podľa okna:')
pprint(m)
print('Makro:', metric_macro_average(m))
print('Mikro:', metric_micro_average(m))

### Hľadanie špičiek spomedzi najbližších susedov vo frekvenčnej doméne 
sínusoida s f = 8 Hz, t = 4 s, fs = 100 Hz v logaritmickej mierke

In [ ]:
rec = {
    'duration': 4,
    'f_sampling': 100,
    'sines': [{'freq': 8, 'amp': 1, 't0': 0, 'tn': 4}],
}
t, y = sine_generate(f=rec['sines'][0]['freq'], t=rec['duration'], fs=rec['f_sampling'])

spectrum = frequency_spectrum(t, y, fs=rec['f_sampling'], window=hann, Wn=127, overlap=0.25, mode='log')
peaks = peaks_indices_windowed(
    peak_find=partial(find_peaks_neighbours, k=3, e=0, h_rel=0, h=0),
    magnitudes=spectrum['magnitudes']
)
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(rec, spectrum['t_windows'], spectrum['resolution'])
found_peak_freqs(truth, peaks, spectrum)

### Generovanie signálu s normálne rozdeleným šumom

In [ ]:
plt.rcParams['figure.figsize'] = (10, 3)
y_noisy = y + np.random.normal(loc=0, scale=0.2, size=len(y))
plt.plot(t, y_noisy)
plt.grid()
plt.show()

### Generátor syntetického signálu so šumom zo sinusioíd
Pravidlá na opis požadovaného signálu musia dodržať nasledujúcu štruktúru:
```json
{
    "duration": 1,
    "f_sampling": 476,
    "sines": [
        {"freq": 5, "amp": 1.5, "t0": 0, "tn": 0.5}
    ],
    "noises": [
        {"amp": 0.2, "t0": 0, "tn": 1.5}
    ]
}
```

In [ ]:
def s_offsets(signal, rules):
    lower = int(signal['t0'] * rules['f_sampling'])
    upper = int((rules['duration'] - signal['tn']) * rules['f_sampling'])
    return lower, upper


def s_duration(signal, rules):
    return int(np.ceil((signal['tn'] - signal['t0']) * rules['f_sampling']))


def signal_rules(filename: str) -> dict:
    with open(filename, 'r') as fsource:
        return json.load(fsource)


def signal_generate(rules: dict) -> TimeSeries:
    N = rules['f_sampling'] * rules['duration']                                         
    Ts = 1 / rules['f_sampling']
    t_series = Ts * np.arange(N)
    signal = np.zeros(N)
    
    for sine in rules.get('sines', []):
        duration_samples = s_duration(sine, rules)
        t_part = Ts * np.arange(duration_samples)
        s = sine['amp'] * np.sin(2 * np.pi * sine['freq'] * t_part)
        signal += np.pad(s, pad_width=s_offsets(sine, rules), constant_values=0)[:len(t_series)]

    for noise in rules.get('noises', []):
        duration_samples = s_duration(noise, rules)
        s = np.random.normal(loc=0, scale=noise['amp'], size=duration_samples)
        signal += np.pad(s, pad_width=s_offsets(noise, rules), constant_values=0)[:len(t_series)]
        
    return t_series, signal


def signal_threshold(y: np.array, level: float) -> np.array:
    return np.where(abs(y) < level, 0, y)

### Vyhladzovanie signálu a oknovaného frekvenčného spektra

In [ ]:
def smooth_signal(t, y, window, k: int):
    kernel = window(k) / k
    return t[k - 1:], np.convolve(y, kernel, 'valid')

def smooth_spectra(spectrum, window, k: int):
    kernel = window(k) / k
    spectrum['bins'] = spectrum['bins'][k - 1:]
    spectrum['magnitudes'] = np.array([
        np.convolve(block, kernel, 'valid') 
        for block in spectrum['magnitudes']
    ])

### Welchova metóda na vyhladzovanie spektrogramov

In [ ]:
def welch_method(spectrum, n=1):
    spectrum['t_windows'] = spectrum['t_windows'][::n]
    averages = []
    
    for i in range(0, len(spectrum['magnitudes']), n):  
        block_sum = np.zeros(len(spectrum['bins']))
        if i+n > len(spectrum['magnitudes']):
            spectrum['t_windows'] = spectrum['t_windows'][:-1]
            break

        for j in range(i, i+n):
            block_sum += spectrum['magnitudes'][j]
        block_sum /= n
        averages.append(block_sum)
        
    spectrum['magnitudes'] = np.array(averages)

### Generátor udalostí
Spektrogram nemáme v online prevádzke dostupný celý naraz, takže detekcia čiar v 2D obrázku neprichádza do úvahy
- `minimal_duration`: po koľkých oknách je vyhlásená špička za udalosť (udáva oneskorenie vyhlásenia štartovacieho eventu)
- `time proximity`: aká veľká môže byť medzera v čase (okná vzad) medzi nájdenými vrcholmi

In [ ]:
def event_detector(bins: np.array, magnitudes: np.array, peaks: np.array, min_duration=1, time_proximity=0) -> list:
    events = []
    for b in range(len(bins)):
        events.append({
            'action': "NONE",
            'start': 0,
            'last_seen': -1,
            'duration': 0,
            'amplitude': 0 
        }) 
    results = []
    
    for t, window in enumerate(peaks):
        y = np.full(len(bins), False)  # maska nájdených vrcholov podľa indexov
        y[window] = True

        for i in range(len(y)):
            if events[i]['action'] == 'END':
                events[i]['start'] = 0,
                events[i]['last_seen'] = -1
                events[i]['duration'] = 0
                events[i]['amplitude'] = 0
    
            events[i]['action'] = 'NONE'
        
            if y[i]:
                events[i]['last_seen'] += 1
                fastforward = max(1, events[i]['last_seen']);

                if (events[i]['duration'] < min_duration and
                        events[i]['duration'] + fastforward >= min_duration):

                    events[i]['action'] = 'START'
                    events[i]['start'] = t - events[i]['duration'] - fastforward + 1;
                
                    results.append({
                        'action': 'START', 
                        'start':  events[i]['start'],
                        'frequency': bins[i],
                        'amplitude': events[i]['amplitude']
                    })

                events[i]['duration'] += fastforward;
                events[i]['amplitude'] += (magnitudes[t][i] - events[i]['amplitude']) / events[i]['duration']
                events[i]['last_seen'] = 0

            elif events[i]['last_seen'] >= 0:
                events[i]['last_seen'] += 1
                
                if events[i]['last_seen'] > time_proximity: 
                    if events[i]['duration'] >= min_duration:
                        events[i]['action'] = 'END'
                        
                        results.append({
                            'action': 'END', 
                            'start':  events[i]['start'],
                            'duration': events[i]['duration'],
                            'frequency': bins[i],
                            'amplitude': events[i]['amplitude']
                        })
                    else:
                        events[i]['start'] = 0
                        events[i]['last_seen'] = -1
                        events[i]['duration'] = 0
                        events[i]['amplitude'] = 0
                
    # Odošli neukončené udalosti stále vo vyrovnávacej pamäti - v online verzii netreba
    for i in range(len(events)):
        
        if events[i]['action'] == 'END':
            events[i]['start'] = 0,
            events[i]['last_seen'] = -1
            events[i]['duration'] = 0
            events[i]['amplitude'] = 0
        
        if events[i]['duration'] >= min_duration:
            results.append({
                'action': 'END', 
                'start':  events[i]['start'],
                'duration': events[i]['duration'],
                'frequency': bins[i],
                'amplitude': events[i]['amplitude']
            })
                
    return results

### Vzorový syntetický signál

In [ ]:
# Prípadne načítanie pravidiel zo súboru
recording = signal_rules('datasets/synth-signal/rec.json')
print(recording)

# Pravidlá na poskladanie tónov
recording = {
    'duration': 5,
    'f_sampling': 476,
    'sines': [
        {'freq': 5, 'amp': 1.5, 't0': 0, 'tn': 1.5},
        {'freq': 18, 'amp': 1.5, 't0': 1.2, 'tn': 5},
        {'freq': 30, 'amp': 1, 't0': 0, 'tn': 5},
        {'freq': 75, 'amp': 2.8, 't0': 0, 'tn': 5},
        {'freq': 230, 'amp': 0.5, 't0': 1, 'tn': 5}
    ],
    'noises': [
        {'amp': 0.2, 't0': 0, 'tn': 5}
    ]
}

# Vzorkovaný signál podľa popisu
t, y = signal_generate(recording)
# Ponechanie vzoriek nad prahovú magnitúdu a vynulovanie ostatných
y_event = signal_threshold(y, 0.8)
# Vyhladenie signálu so zvolenou okovou funkciou a veľkosťou kernelu
t_s, y_s = smooth_signal(t, y_event, boxcar, k=10)

# Vykreslenie grafu
plt.rcParams['figure.figsize'] = (20, 10)
fig, axes = plt.subplots(3, 1)
axes[0].set_title('Syntetický signál so šumom')
axes[0].set_ylabel('Amplitúda')
axes[0].plot(t, y)
axes[0].grid()

axes[1].set_title('Ponechanie najvyšších magnitúd y > 0.8')
axes[1].plot(t, y_event)
axes[1].set_ylabel('Amplitúda')
axes[1].grid()

axes[2].set_title('Vyhladenie udalostí')
axes[2].plot(t_s, y_s)
axes[2].set_ylabel('Amplitúda')
axes[2].set_xlabel('Čas [s]')
axes[2].grid()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (20, 5)
fig, axes = plt.subplots(1, 3)
axes[0].hist(y, bins=100)
axes[1].hist(y_event, bins=100)
axes[2].hist(y_s, bins=100)
plt.show()

### Detekcia špičiek v signále so šumom manuálnym ladením hyperparametrov

In [ ]:
t, y = signal_generate(recording)
y = signal_threshold(y, 0.8) 

spectrum = frequency_spectrum(t, y, fs=recording['f_sampling'], window=hann, Wn=511, overlap=0.5, mode='log')
smooth_spectra(spectrum, boxcar, 4)

func = partial(find_peaks_neighbours, k=10, e=0, h_rel=15, h=0)
# func = partial(find_peaks_zero_crossing, k=1, slope=8)
# func = partial(find_peaks_hill_walker, tolerance=0, hole=0, prominence=30, isolation=5)

peaks = peaks_indices_windowed(func, spectrum['magnitudes'])
plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

truth = ground_truth(recording, spectrum['t_windows'], spectrum['resolution'])
m = metrics(spectrum['bins'], truth, peaks)

print('Macro:', metric_macro_average(m))
print('Micro:', metric_micro_average(m))

### Detektor udalostí testovaný na syntetických dátach

In [ ]:
events = event_detector(spectrum['bins'], spectrum['magnitudes'], peaks, min_duration=2, time_proximity=2)
events

### Vizualizácia udalostí v spektrograme

In [ ]:
plot_spectra_events(spectrum, events)

### Úspešnosť klasifikátora na trénovacej a testovacej množine
Spôsob hľadania najlepších parametrov je grid search na trénovacich dátach

In [ ]:
def evaluation(bins, truth, peaks, mode):
    if mode == 'micro':
        return metric_micro_average(metrics(bins, truth, peaks))
    elif mode == 'macro':
        return metric_macro_average(metrics(bins, truth, peaks))

def grid_search(spectrum, truth, algorithm, mode, *args):
    best = None
    accuracy = 0
    for params in np.array(np.meshgrid(*args)).T.reshape(-1, len(args)):
        m = evaluation(spectrum['bins'], truth, algorithm(spectrum, *params), mode)
        if m['ACC'] > accuracy:
            accuracy = m['ACC']
            best = params
    return best

In [ ]:
rec_train = {
    'duration': 30,
    'f_sampling': 128,
    'sines': [
        {'freq': 5, 'amp': 1.8, 't0': 0, 'tn': 10},
        {'freq': 12, 'amp': 2.3, 't0': 5, 'tn': 25},
        {'freq': 45, 'amp': 0.8, 't0': 0, 'tn': 30},
    ],
    'noises': [
        {'amp': 0.1, 't0': 0, 'tn': 5}
    ]
}
rec_test = {
    'duration': 8,
    'f_sampling': 128,
    'sines': [
        {'freq': 8, 'amp': 1.6, 't0': 5, 'tn': 8},
        {'freq': 24, 'amp': 2.7, 't0': 0, 'tn': 6},
        {'freq': 32, 'amp': 1.1, 't0': 0, 'tn': 8},
    ],
    'noises': [
        {'amp': 0.1, 't0': 0, 'tn': 5}
    ]
}
t_train, y_train = signal_generate(rec_train)
t_test, y_test = signal_generate(rec_test)

train_spectra = frequency_spectrum(
    t_train, y_train, fs=rec_train['f_sampling'], window=hann, Wn=255, overlap=0.5, mode='linear'
)
test_spectra = frequency_spectrum(
    t_test, y_test, fs=rec_test['f_sampling'], window=hann, Wn=255, overlap=0.5, mode='linear'
)

train_labels = ground_truth(
    rec_train, train_spectra['t_windows'], train_spectra['resolution']
)
test_labels = ground_truth(
    rec_test, test_spectra['t_windows'], test_spectra['resolution']
)

### Úspešnosť: význačnosť vrchola spomedzi susedov

In [ ]:
def algo_no1(spectrum, k, e, h_rel):
    f = partial(find_peaks_neighbours, k=int(k), e=int(e), h_rel=h_rel, h=0)
    return peaks_indices_windowed(f, spectrum['magnitudes'])

# Možno previesť do relatívnych čísel vzhľadom na magnitúdy v okne
n = 16 # len(train_spectra['bins']) // 4
K = np.arange(3, n, 3)
E = np.linspace(0, np.max(train_spectra['magnitudes']), 3)  
H = np.linspace(0, np.max(train_spectra['magnitudes']), 8)

best = grid_search(train_spectra, train_labels, algo_no1, 'macro', K, E, H)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no1(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no1(test_spectra, *best), 'macro')

print('k =', ', '.join([f'{x}' for x in K]))
print('e =', ', '.join([f'{x:.2f}' for x in E]))
print('h_rel =', ', '.join([f'{x:.2f}' for x in H]))
print()
print('Best k =', best[0])
print('Best e =', best[1])
print('Best h_rel =', best[2])
print()
print('Train:', m_train)
print('Test:', m_test)

### Úspešnosť: algoritmus prechodu nulou do záporu

In [ ]:
def algo_no2(spectrum, k, s):
    f = partial(find_peaks_zero_crossing, k=int(k), slope=s)
    return peaks_indices_windowed(f, spectrum['magnitudes'])


K = np.arange(1, 50)
S = np.linspace(0, 6, 10)
print('k=', K); print('slope=', S)


best = grid_search(train_spectra, train_labels, algo_no2, 'macro', K, S)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no2(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no2(test_spectra, *best), 'macro')

print('k =', ', '.join([f'{x}' for x in K]))
print('s =', ', '.join([f'{x:.2f}' for x in S]))
print()
print('Best k =', best[0])
print('Best s =', best[1])
print()
print('Train:', m_train)
print('Test:', m_test)

### Úspešnosť: algoritmus horského turistu

In [ ]:
def algo_no3(spectrum, t, h, p, i):
    f = partial(find_peaks_hill_walker, tolerance=t, hole=int(h), prominence=p, isolation=int(i))
    return peaks_indices_windowed(f, spectrum['magnitudes'])

H = np.linspace(0, np.max(spectrum['magnitudes']), 6)
L = np.linspace(0, len(spectrum['bins']), 6)
print('tolerance=', H); print('hole=', L); print('prominence=', H); print('isolation=', L)


best = grid_search(train_spectra, train_labels, algo_no3, 'macro', H, L, H, L)
m_train = evaluation(train_spectra['bins'], train_labels, algo_no3(train_spectra, *best), 'macro')
m_test = evaluation(test_spectra['bins'], test_labels, algo_no3(test_spectra, *best), 'macro')

print('h =', ', '.join([f'{x}' for x in K]))
print('l =', ', '.join([f'{x:.2f}' for x in S]))
print()
print('Best tolerance =', best[0])
print('Best hole =', best[1])
print('Best prominence =', best[2])
print('Best isolation =', best[3])
print()
print('Train:', m_train)
print('Test:', m_test)

### Hľadanie špičiek vo vlastných datasetoch

In [ ]:
from utils import load_dataset

vibrations, FS_HZ = load_dataset('esp', 'L83_4940_Alexyho_Svantnerova.csv')
vibrations.info()

In [ ]:
a = 20000
b = 26000
t = vibrations.index.array[a:b]
y = vibrations.z.array[a:b]

y = signal_threshold(y - y.mean(), 1)
#t, y = smooth_signal(t, y, boxcar, 32)
plt.rcParams['figure.figsize'] = (20, 4)
plt.plot(t, y)

In [ ]:
spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=512, overlap=0.5, mode='linear')
# welch_method(spectrum, 3)
# smooth_spectra(spectrum, boxcar, 10)

# func = partial(find_peaks_threshold, t=-15)
func = partial(find_peaks_neighbours, k=20, e=0, h_rel=5, h=0)
# func = partial(find_peaks_zero_crossing, k=30, slope=10)
# func = partial(find_peaks_hill_walker, tolerance=5, hole=0, prominence=10, isolation=5)
peaks = peaks_indices_windowed(func, spectrum['magnitudes'])

plot_spectra_slices(spectrum['bins'], spectrum['magnitudes'], peaks)

### Odčítanie bežiaceho priemeru 

- online aproximácia pre: `y - y.mean()`
- Cumulative moving average: https://en.wikipedia.org/wiki/Moving_average#Cumulative_moving_average
- Welfordov algoritmus (2.20 v BP)

In [ ]:
def filter_running_average(y: np.array):
    avg = y[0]
    n = 1

    for i, sample in enumerate(y[1:], start=1):
        n += 1
        avg += (sample - avg) / n 
        y[i] -= avg
        
    return y

### Demonštrácia pipeline

In [ ]:
# Získanie signálu v časovej doméne
t = vibrations.index.array
y = vibrations.z.array

# Odčítanie bežiaceho priemeru (online aproximácia pre: y - y.mean())
# y = filter_running_average(y)

# Orezanie vzoriek s nízkou magnitúdou
# y = signal_threshold(y, 1)

# Vyhladenie signálu v času
# t, y = smooth_signal(t, y, boxcar, 4)

# Frekvenčná transformácia (fft, rfft, dct-ii, dct-iv)
spectrum = frequency_spectrum(t, y, fs=FS_HZ, window=hann, Wn=256, overlap=0.5, trans='dct-iv', mode='linear')

# Vyhadenie spektier
# smooth_spectra(spectrum, boxcar, 3)

# Welchova metóda priemerovanie n spektrogramov
welch_method(spectrum, n=3)

# Hľadanie špičiek v spektre
# func = partial(find_peaks_threshold, t=-15)
func = partial(find_peaks_neighbours, k=10, e=0, h_rel=0, h=0)
# func = partial(find_peaks_zero_crossing, k=5, slope=6)
# func = partial(find_peaks_hill_walker, tolerance=10, hole=1, prominence=8, isolation=5)
peaks = peaks_indices_windowed(func, spectrum['magnitudes'])

# Nakresli spektrogram
print(spectrum['t_windows'].shape)
print(spectrum['bins'].shape)
print(spectrum['magnitudes'].shape)
plot_spectra_heatmap(spectrum, peaks) # ylog=True, yrange=(0,50))

In [ ]:
# Zisti či sa jedná sa o udalosť podľa tolerancie trvania
events = event_detector(spectrum['bins'], spectrum['magnitudes'], peaks, min_duration=3, time_proximity=5)
"""
for event in events:
    if event['action'] == "END":
        print(event['frequency'], event['start'], event['start'] + event['duration'])
"""
plot_spectra_events(spectrum, events)